
'''
* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
'''''

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mping
import os.path as path
import collections
import os
from moviepy.editor import VideoFileClip
import pickle
%matplotlib qt
ym_per_pix = 30/720
xm_per_pix = 3.7/700
time_window = 10
#=======================================================================================
# 函数名称：camera_calibrae
# 函数功能：计算相机矩阵
# 输入参数：棋盘格图片的存储文件夹，vebose
# 返回值：相机参数
# 备注信息：
#=======================================================================================
def lazy_calibration(func):
    calibration_cache = 'camera_cal/calibraion_data.pickle'
    def wrapper(*args,**kwargs):
        if path.exists(calibration_cache):
            print('Load cached camera calibraion...',end=' ')
            with open(calibration_cache,'rb') as dump_file:
                calibration = pickle.load(dump_file)
        else:
            print('Computing camera calibaraion...',end = ' ')
            calibration = func(*args,**kwargs)
            with open(calibration_cache,'wb') as dump_file:
                pickle.dump(calibration,dump_file)
        print('done')
        return calibration
    return wrapper
@lazy_calibration
def camera_calibrate(camera_cal_direction,distory_status = False):
    assert path.exists(camera_cal_direction),'"{}" must exist and contain calibration images'.format(camera_cal_direction)
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
    
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    #images = glob.glob('./camera_cal/calibration*.jpg')
    images = glob.glob(path.join(camera_cal_direction,'calibration*.jpg'))
    # Step through the list and search for chessboard corners
    for fname in images:
        img = mping.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            cv2.imshow('img',img)
            cv2.waitKey(50)
    
    #if distory_status:
    cv2.destroyAllWindows()
    ret,mtx,dist,rvecs,tvecs = cv2.calibrateCamera(objpoints,imgpoints,gray.shape[::-1],None,None)#相机矩阵计算
    #print(gray.shape)
    #print(gray.shape[::-1])
    return ret,mtx,dist,rvecs,tvecs

In [2]:
#=======================================================================================
# 函数名称：undistort
# 函数功能：图片去畸变
# 输入参数：畸变图片，相机参数，vebose
# 返回值：无畸变图片
# 备注信息：
#=======================================================================================
def undistort(disroted_image,mtx,dist,show_status = False):
    undistorted_image = cv2.undistort(disroted_image,mtx,dist,None,mtx)
    if show_status:
        fig,ax = plt.subplots(nrows=1,ncols=2)
        ax[0].imshow(disroted_image)
        ax[1].imshow(undistorted_image)
        plt.show()
    return undistorted_image

In [17]:
# 读取样本图片并且可视化去畸变效果
%matplotlib qt
#ret,mtx,dist,rvecs,tvecs = camera_calibrate(camera_cal_direction='camera_cal')
#print('rvecs',rvecs)
#print('tvecs',tvecs)
img_test = mping.imread('camera_cal/donottestcalibration1.jpg')
print(img_test.shape)
#img_test_undistorted = undistort(img_test,mtx,dist,True)


(720, 1280, 3)


In [4]:
#=======================================================================================
# 函数名称：threshold_binary
# 函数功能：根据HLS空间，梯度以及直方图获得车道线
# 输入参数：图片，阈值，vebose
# 返回值：二值化图像
# 备注信息：
#=======================================================================================
# 该框用于计算生成二值化图像：HSV空间 + Gradient + 直方图
#S_min = np.array([0,70,70])
#LS_max = np.array([50,255,255])
def threshold_binary(img,s_thresh = (170,255),gradient_thesh = (20,100),show_status = False):
    hls = cv2.cvtColor(img,cv2.COLOR_RGB2HLS)
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    #Sobel:
    #Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1)
    #Calculate the magnitude
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    abs_sobelxy = np.sqrt(abs_sobelx**2+abs_sobely**2)
    #Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobelxy = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    _,sobel_img = cv2.threshold(scaled_sobelxy,gradient_thesh[0],gradient_thesh[1],cv2.THRESH_BINARY)
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    #Histogram
    eq_img = cv2.equalizeHist(gray)
    _,th = cv2.threshold(eq_img,thresh=250,maxval=255,type=cv2.THRESH_BINARY)
    
    binary_img = np.zeros_like(gray)
    binary_img = np.logical_or(binary_img,sobel_img)
    binary_img = np.logical_or(binary_img,s_binary)
    binary_img = np.logical_or(binary_img,th)
    kernel = np.ones((5,5),np.uint8)
    binary_img = cv2.morphologyEx(binary_img.astype(np.uint8),cv2.MORPH_CLOSE,kernel)
    if show_status:
        fig,ax = plt.subplots(nrows=1,ncols=2)
        ax[0].imshow(img)
        ax[1].imshow(binary_img,cmap='gray')#(cv2.cvtColor(binary_img,cv2.THRESH_BINARY))
        plt.show()
    return binary_img

In [5]:
#test_images = glob.glob('test_images/test*.jpg')
#test_image = test_images[4]
#test_image = mping.imread(test_image)
#cv2.imshow(test_image)
#img_test_undistorted = undistort(test_image,mtx,dist,True)
#binary_img = threshold_binary(img_test_undistorted,s_thresh = (170,255),gradient_thesh = (20,100),show_status=True)

In [6]:
#=======================================================================================
# 函数名称：bird_transform
# 函数功能：透视变换
# 输入参数：图片，，vebose
# 返回值：鸟瞰图
# 备注信息：
#=======================================================================================
def bird_tranform(img,show_status = False):
    height,width = img.shape[:2]
    src = np.float32([[width,height-10],
                   [0,height-10],
                   [546,460],
                   [732,460]])
    dst = np.float32([[width,height],
                   [0,height],
                   [0,0],
                   [width,0]])
    M = cv2.getPerspectiveTransform(src,dst)
    Minv = cv2.getPerspectiveTransform(dst,src)
    warped = cv2.warpPerspective(img,M,(width,height),flags = cv2.INTER_LINEAR)
    if show_status:
        fig,ax = plt.subplots(nrows=1,ncols=2)
        ax[0].imshow(img)
        ax[1].imshow(warped)#(cv2.cvtColor(binary_img,cv2.THRESH_BINARY))
        plt.show()
    return warped,M,Minv

In [7]:
#test_images = glob.glob('test_images/test*.jpg')
#test_image = test_images[4]
#test_image = mping.imread(test_image)
#te2 = mping.imread('test_images/straight_lines1.jpg')
#print(type(test_image))
#cv2.imshow(test_image)
#img_test_undistorted = undistort(test_image,mtx,dist,True)
#binary_img = threshold_binary(img_test_undistorted,s_thresh = (170,255),gradient_thesh = (20,100),show_status=True)
#print(type(binary_img))
#bird_img,M,Minv = bird_tranform(te2,True)

In [8]:
class Line:
    def __init__(self,buffer_len = 10):
        self.datected = False
        self.last_fit_pixel = None
        self.last_fit_meter = None
        self.recent_fits_pixel = collections.deque(maxlen = buffer_len)# 创建队列
        self.recent_fits_meter = collections.deque(maxlen = buffer_len * 2)
        self.radius_of_curvature = None
        self.all_x = None
        self.all_y  = None
    def update_line(self,new_fit_pixel,new_fit_meter,detected,clear_buffer = False):
        self.detected = detected
        if clear_buffer:
            self.recent_fits_pixel = []
            sefl.recent_fits_meter = []
        self.last_fit_pixel = new_fit_pixel
        self.last_fit_meter = new_fit_meter
        self.recent_fits_pixel.append(self.last_fit_pixel)
        self.recent_fits_meter.append(self.last_fit_meter)
    def draw(self,mask,color = (255,0,0),line_width = 50,average = False):
        h,w,c = mask.shape
        ploty = np.linspace(0,h-1,h)
        coeffs = self.average_fit if average else self.last_fit_pixel
        
        line_center = coeffs[0] * ploty**2 + coeffs[1] * ploty + coeffs[2]
        line_left_side = line_center - line_width//2
        line_right_side = line_center + line_width//2
        
        pts_left = np.array(list(zip(line_left_side,ploty)))
        pts_right = np.array(np.flipud(list(zip(line_right_side,ploty))))
        pts = np.vstack([pts_left,pts_right])
        return cv2.fillPoly(mask,[np.int32(pts)],color)
    
    @property
    def average_fit(self):
        return np.mean(self.recent_fits_pixel,axis = 0)
    @property
    def curvature(self):
        y_eval = 0
        coeffs = self.average_fit
        return ((1+(2*coeffs[0]*y_eval + coeffs[1]**2)**1.5)/np.absolute(2*coeffs[0]))
    @property
    def curvature_meter(self):
        y_eval = 0
        coeffs = np.mean(self.recent_fits_meter,axis = 0)
        return ((1+(2*coeffs[0]*y_eval + coeffs[1]**2)**1.5)/np.absolute(2*coeffs[0]))        
        

In [9]:
def get_fits_by_sliding(birdeye_binary,line_lt,line_rt,n_windows=9,verbose=False):
    height,width = birdeye_binary.shape
    margin = 100
    minpix = 50
    histogram = np.sum(birdeye_binary[height//2:-30,:],axis=0)
    out_img = np.dstack((birdeye_binary,birdeye_binary,birdeye_binary))*255
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    window_height = np.int(birdeye_binary.shape[0]//n_windows)
    nonzero = birdeye_binary.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    leftx_current = leftx_base
    rightx_current = rightx_base
    left_lane_inds = []
    right_lane_inds = []
    # Step through the windows one by one
    for window in range(n_windows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = birdeye_binary.shape[0] - (window+1)*window_height
        #print('win_y_low',win_y_low)
        win_y_high = birdeye_binary.shape[0] - window*window_height
        #print('win_y_high',win_y_high)
        #print('leftx_current',leftx_current)
        #print('rightx_current',rightx_current)
        win_xleft_low = leftx_current - margin  # Update this横向
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
        # 当前窗口内的非零元素坐标
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass
    line_lt.all_x = nonzerox[left_lane_inds]
    line_lt.all_y = nonzeroy[left_lane_inds] 
    line_rt.all_x = nonzerox[right_lane_inds]
    line_rt.all_y = nonzeroy[right_lane_inds]
    detected = True
    if not list(line_lt.all_x) or not list(line_lt.all_y):
        left_fit_pixel = line_lt.last_fit_pixel
        left_fit_meter = line_lt.last_fit_meter
        detected = False
    else:
        left_fit_pixel = np.polyfit(line_lt.all_y,line_lt.all_x,2)
        left_fit_meter = np.polyfit(line_lt.all_y * ym_per_pix,line_lt.all_x * xm_per_pix,2)
    if not list(line_rt.all_x) or not list(line_rt.all_y):
        right_fit_pixel = line_rt.last_fit_pixel
        right_fit_meter = line_rt.last_fit_meter
        detected = False
    else:
        right_fit_pixel = np.polyfit(line_rt.all_y,line_rt.all_x,2)
        right_fit_meter = np.polyfit(line_rt.all_y * ym_per_pix,line_rt.all_x * xm_per_pix,2)
    line_lt.update_line(left_fit_pixel,left_fit_meter,detected = detected)
    line_rt.update_line(right_fit_pixel,right_fit_meter,detected = detected)
    ploty = np.linspace(0,height-1,height)
    left_fitx = left_fit_pixel[0]*ploty**2 + left_fit_pixel[1]*ploty + left_fit_pixel[2]
    right_fitx = right_fit_pixel[0]*ploty**2 + right_fit_pixel[1]*ploty + right_fit_pixel[2]
    out_img[nonzeroy[left_lane_inds],nonzerox[left_lane_inds]] = [255,0,0]
    out_img[nonzeroy[right_lane_inds],nonzerox[right_lane_inds]] = [0,0,255]
    if verbose:
        f,ax = plt.subplots(1,2)
        f.set_facecolor('white')
        ax[0].imshow(birdeye_binary,cmap = 'gray')
        ax[1].imshow(out_img)
        ax[1].plot(left_fitx,ploty,color = 'yellow')
        ax[1].plot(right_fitx,ploty,color = 'yellow')
        ax[1].set_xlim(0,1200)
        ax[1].set_ylim(720,0)
        plt.show
    return line_lt,line_rt,out_img
    

In [10]:
def get_fits_by_previous(birdeye_binary,line_lt,line_rt,verbose=False):
    height,width = birdeye_binary.shape
    margin = 100
    #minpix = 50
    #histogram = np.sum(birdeye_binary[height//2:-30,:],axis=0)
    #out_img = np.dstack((birdeye_binary,birdeye_binary,birdeye_binary))*255
    #midpoint = np.int(histogram.shape[0]//2)
    #leftx_base = np.argmax(histogram[:midpoint])
    #rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    #window_height = np.int(birdeye_binary.shape[0]//n_windows)
    nonzero = birdeye_binary.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    #leftx_current = leftx_base
    #rightx_current = rightx_base
    #left_lane_inds = []
    #right_lane_inds = []
    # Step through the windows one by one
    #for window in range(n_windows):
        # Identify window boundaries in x and y (and right and left)
        #win_y_low = birdeye_binary.shape[0] - (window+1)*window_height
        #print('win_y_low',win_y_low)
        #win_y_high = birdeye_binary.shape[0] - window*window_height
        #print('win_y_high',win_y_high)
        #print('leftx_current',leftx_current)
        #print('rightx_current',rightx_current)
        #win_xleft_low = leftx_current - margin  # Update this横向
        #win_xleft_high = leftx_current + margin  # Update this
        #win_xright_low = rightx_current - margin  # Update this
        #win_xright_high = rightx_current + margin  # Update this
    left_fit_pixel = line_lt.last_fit_pixel
    left_fit_meter = line_lt.last_fit_meter
    right_fit_pixel = line_rt.last_fit_pixel
    right_fit_meter = line_rt.last_fit_meter
    left_lane_inds = ((nonzerox > (left_fit_pixel[0]*(nonzeroy**2)+left_fit_pixel[1]*nonzeroy+left_fit_pixel[2]-margin))&
                      (nonzerox < (left_fit_pixel[0]*(nonzeroy**2)+left_fit_pixel[1]*nonzeroy+left_fit_pixel[2]+margin)))
    right_lane_inds = ((nonzerox > (right_fit_pixel[0]*(nonzeroy**2)+right_fit_pixel[1]*nonzeroy+right_fit_pixel[2]-margin))&
                       (nonzerox < (right_fit_pixel[0]*(nonzeroy**2)+right_fit_pixel[1]*nonzeroy+right_fit_pixel[2]+margin)))
    line_lt.all_x = nonzerox[left_lane_inds]
    line_lt.all_y = nonzeroy[left_lane_inds] 
    line_rt.all_x = nonzerox[right_lane_inds]
    line_rt.all_y = nonzeroy[right_lane_inds]
    
    # Draw the windows on the visualization image
    #cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
    #cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
    # 当前窗口内的非零元素坐标
    #good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
    #good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
    # Append these indices to the lists
    #left_lane_inds.append(good_left_inds)
    #right_lane_inds.append(good_right_inds)
        
    #if len(good_left_inds) > minpix:
    #    leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
    #if len(good_right_inds) > minpix:
    #    rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    #try:
    #    left_lane_inds = np.concatenate(left_lane_inds)
    #    right_lane_inds = np.concatenate(right_lane_inds)
    #except ValueError:
    #    # Avoids an error if the above is not implemented fully
    #    pass

    detected = True
    if not list(line_lt.all_x) or not list(line_lt.all_y):
        left_fit_pixel = line_lt.last_fit_pixel
        left_fit_meter = line_lt.last_fit_meter
        detected = False
    else:
        left_fit_pixel = np.polyfit(line_lt.all_y,line_lt.all_x,2)
        left_fit_meter = np.polyfit(line_lt.all_y * ym_per_pix,line_lt.all_x * xm_per_pix,2)
    if not list(line_rt.all_x) or not list(line_rt.all_y):
        right_fit_pixel = line_rt.last_fit_pixel
        right_fit_meter = line_rt.last_fit_meter
        detected = False
    else:
        right_fit_pixel = np.polyfit(line_rt.all_y,line_rt.all_x,2)
        right_fit_meter = np.polyfit(line_rt.all_y * ym_per_pix,line_rt.all_x * xm_per_pix,2)
    line_lt.update_line(left_fit_pixel,left_fit_meter,detected = detected)
    line_rt.update_line(right_fit_pixel,right_fit_meter,detected = detected)
    
    ploty = np.linspace(0,height-1,height)
    left_fitx = left_fit_pixel[0]*ploty**2 + left_fit_pixel[1]*ploty + left_fit_pixel[2]
    right_fitx = right_fit_pixel[0]*ploty**2 + right_fit_pixel[1]*ploty + right_fit_pixel[2]
    
    out_img = np.dstack((birdeye_binary, birdeye_binary, birdeye_binary))*255
    window_img = np.zeros_like(out_img)
    out_img[nonzeroy[left_lane_inds],nonzerox[left_lane_inds]] = [255,0,0]
    out_img[nonzeroy[right_lane_inds],nonzerox[right_lane_inds]] = [0,0,255]
    
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    

    if verbose:
        f,ax = plt.subplots(1,2)
        f.set_facecolor('white')
        ax[0].imshow(birdeye_binary,cmap = 'gray')
        ax[1].imshow(result)
        ax[1].plot(left_fitx,ploty,color = 'yellow')
        ax[1].plot(right_fitx,ploty,color = 'yellow')
        ax[1].set_xlim(0,1200)
        ax[1].set_ylim(720,0)
        plt.show
    return line_lt,line_rt,out_img
    

In [11]:
def draw_back_onto_the_road(img_undistorted,Minv,line_lt,line_rt,keep_state):
    height,width,_ = img_undistorted.shape
    left_fit = line_lt.average_fit if keep_state else line_lt.last_fit_pixel
    right_fit = line_rt.average_fit if keep_state else line_rt.last_fit_pixel
    
    ploty = np.linspace(0,height-1,height)
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    road_warp = np.zeros_like(img_undistorted,dtype=np.uint8)
    pts_left = np.array([np.transpose(np.vstack([left_fitx,ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx,ploty])))])
    pts = np.hstack((pts_left,pts_right))
    cv2.fillPoly(road_warp,np.int_([pts]),(0,255,0))
    road_dewarped = cv2.warpPerspective(road_warp,Minv,(width,height))
    blend_onot_road = cv2.addWeighted(img_undistorted,1.0,road_dewarped,0.3,0)
    
    line_warp = np.zeros_like(img_undistorted)
    line_warp = line_lt.draw(line_warp,color = (255,0,0),average = keep_state)
    line_warp = line_rt.draw(line_warp,color = (0,0,255),average = keep_state)
    line_dewarped = cv2.warpPerspective(line_warp,Minv,(width,height))
    
    lines_mask = blend_onot_road.copy()
    idx = np.any([line_dewarped != 0][0],axis=2)
    lines_mask[idx] = line_dewarped[idx]
    
    blend_onot_road = cv2.addWeighted(lines_mask,0.8,blend_onot_road,0.5,0)
    
    return blend_onot_road

In [12]:
line_lt = Line(buffer_len=10)
line_rt = Line(buffer_len=10)
#test_images = glob.glob('test_images/test*.jpg')
#test_image = test_images[4]
#test_image = mping.imread(test_image)
#te2 = mping.imread('test_images/straight_lines1.jpg')
#print(type(test_image))
#cv2.imshow(test_image)
#img_test_undistorted = undistort(test_image,mtx,dist,False)
#binary_img = threshold_binary(img_test_undistorted,s_thresh = (170,255),gradient_thesh = (20,100),show_status=False)
#print(type(binary_img))
#bird_img,M,Minv = bird_tranform(binary_img,False)
#line_lt,line_rt,img_out = get_fits_by_sliding(bird_img,line_lt,line_rt,n_windows=7,verbose = True)

In [13]:
def compute_offset_from_center(line_lt,line_rt,frame_width):
    if line_lt.detected and line_rt.detected:
        line_lt_bottom = np.mean(line_lt.all_x[line_lt.all_y > 0.95 * line_lt.all_y.max()])
        line_rt_bottom = np.mean(line_rt.all_x[line_rt.all_y > 0.95 * line_rt.all_y.max()])
        lane_width = line_rt_bottom - line_lt_bottom
        midpoint = frame_width/2
        offset_pix = abs((line_lt_bottom + lane_width/2)-midpoint)
        offset_meter = xm_per_pix * offset_pix
    else:
        offset_meter = -1
    return offset_meter

In [14]:
def prepare_out_blend_frame(blend_on_road,img_binary,img_birdeye,img_fit,line_lt,line_rt,offset_meter):
    h,w = blend_on_road.shape[:2]
    # 在同一个小图里面绘制小窗口
    thumb_ratio = 0.2
    thumb_h,thumb_w = int(thumb_ratio * h),int(thumb_ratio * w)
    # 小窗口的位置偏移
    off_x = 20
    off_y = 15
    
    mask = blend_on_road.copy()
    mask = cv2.rectangle(mask,pt1 = (0,0),pt2 = (w,thumb_h + 2*off_y),color = (0,0,0),thickness = cv2.FILLED)
    blend_on_road = cv2.addWeighted(src1 = mask,alpha = 0.2,src2=blend_on_road,beta=0.8,gamma = 0)
    
    thumb_binary = cv2.resize(img_binary,dsize=(thumb_w,thumb_h))
    thumb_binary = np.dstack([thumb_binary,thumb_binary,thumb_binary])*255
    blend_on_road[off_y:thumb_h+off_y,off_x:off_x+thumb_w,:] = thumb_binary
    
    thumb_birdeye = cv2.resize(img_birdeye,dsize = (thumb_w,thumb_h))
    thumb_birdeye = np.dstack([thumb_birdeye,thumb_birdeye,thumb_birdeye])*255
    blend_on_road[off_y:thumb_h+off_y,2*off_x+thumb_w:2*(off_x+thumb_w),:] = thumb_birdeye
    
    thumb_img_fit = cv2.resize(img_fit,dsize = (thumb_w,thumb_h))
    #thumb_img_fit = np.dstack([thumb_img_fit,thumb_img_fit,thumb_img_fit])*255
    blend_on_road[off_y:thumb_h+off_y,3*off_x+2*thumb_w:3*(off_x+thumb_w),:] = thumb_img_fit
    
    mean_curvature_meter = np.mean([line_lt.curvature_meter,line_rt.curvature_meter])
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(blend_on_road,'Curvature Radius: {:.02f}m'.format(mean_curvature_meter),(860,60),font,0.9,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(blend_on_road,'Offset from center: {:.02f}m'.format(offset_meter),(860,130),font,0.9,(255,255,255),2,cv2.LINE_AA)
    
    return blend_on_road

In [15]:
processed_frames = 0
def pipline_algorithm(frame,keep_state=True):
    global line_lt,line_rt,processed_frames
    
    img_undistorted = undistort(frame,mtx,dist,False)
    binary_img = threshold_binary(img_undistorted,s_thresh = (170,255),gradient_thesh = (20,100),show_status=False)
    #print(type(binary_img))
    bird_img,M,Minv = bird_tranform(binary_img,False)
    if processed_frames > 0 and keep_state and line_lt.detected and line_rt.detected:
        line_lt,line_rt,img_out = get_fits_by_previous(bird_img,line_lt,line_rt,verbose = False)
    else:
        line_lt,line_rt,img_out = get_fits_by_sliding(bird_img,line_lt,line_rt,n_windows=7,verbose = False)
    offset_meter = compute_offset_from_center(line_lt,line_rt,frame_width = frame.shape[1])
    blend_on_road = draw_back_onto_the_road(img_undistorted,Minv,line_lt,line_rt,keep_state)
    blend_output = prepare_out_blend_frame(blend_on_road,binary_img,bird_img,img_out,line_lt,line_rt,offset_meter)
    processed_frames+=1
    return blend_output


In [16]:

ret,mtx,dist,rvecs,tvecs = camera_calibrate(calib_images_dir='camera_cal')
mode = 'video'
if mode == 'video':
    selector = 'project'
    clip = VideoFileClip('{}_video.mp4'.format(selector)).fl_image(pipline_algorithm)
    clip.write_videofile('out_{}_{}.mp4'.format(selector,time_window),audio=False)
else:
    test_img_dir = 'test_images'
    for test_img in os.listdir(test_img_dir):
        frame = cv2.imread(path.join(test_img_dir,test_img))
        blend = pipline_algorithm(frame,keep_state = False)
        cv2.imwrite('output_images/{}'.format(test_img),blend)
        plt.imshow(cv2.cvtColor(blend,code = cv2.COLOR_BGR2RGB))
        plt.show()

Load cached camera calibraion... done
[MoviePy] >>>> Building video out_project_10.mp4
[MoviePy] Writing video out_project_10.mp4


100%|█████████▉| 1260/1261 [02:51<00:00,  7.43it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: out_project_10.mp4 

